In [1]:
import numpy as np
import pandas as pd

train_data = pd.read_csv('amazon_train.csv')

# نمایش چند خط اول داده‌ها برای بررسی
print(train_data.head())

In [2]:
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

# کدگذاری شناسه‌های کاربر و محصول
train_data['UserEnc'] = user_encoder.fit_transform(train_data['UserID'])
train_data['ProductEnc'] = item_encoder.fit_transform(train_data['ProductID'])

# نمایش چند خط اول داده‌ها برای بررسی
print(train_data.head())

In [3]:
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns

# بررسی وجود NaN در ستون rating
print("تعداد مقادیر NaN در ستون Rating:", train_data['Rating'].isna().sum())

# حذف ردیف‌های حاوی NaN در ستون rating
train_data = train_data.dropna(subset=['Rating'])

# بررسی دوباره برای اطمینان از حذف NaN
print("تعداد مقادیر NaN پس از حذف:", train_data['Rating'].isna().sum())

# بررسی توازن مجموعه‌داده (توزیع امتیازات در ستون rating)
print("\nتوزیع اولیه امتیازات:")
print(train_data['Rating'].value_counts())

# نمایش توزیع امتیازات با نمودار
plt.figure(figsize=(8, 5))
sns.countplot(x='Rating', data=train_data)
plt.title('توزیع امتیازات در مجموعه‌داده')
plt.xlabel('امتیاز')
plt.ylabel('تعداد')
plt.show()

# نمونه‌افزایی تصادفی برای متوازن‌تر کردن مجموعه‌داده
# تعریف استراتژی نمونه‌افزایی (به‌صورت دستی تنظیم می‌کنیم تا تفاوت چشم‌گیر نباشد)
sampling_strategy = {1: int(train_data['Rating'].value_counts().max() * 0.5),  # برای دسته‌های کوچک‌تر
                    2: int(train_data['Rating'].value_counts().max() * 0.6),
                    3: int(train_data['Rating'].value_counts().max() * 0.7),
                    4: int(train_data['Rating'].value_counts().max() * 0.8),
                    5: train_data['Rating'].value_counts().max()}  # دسته اکثریت بدون تغییر

ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X = train_data[['UserEnc', 'ProductEnc']]  # ویژگی‌های ورودی
y = train_data['Rating']  # برچسب‌ها
X_resampled, y_resampled = ros.fit_resample(X, y)

# ایجاد DataFrame جدید با داده‌های نمونه‌افزایی‌شده
train_data_balanced = pd.DataFrame({
    'UserID': user_encoder.inverse_transform(X_resampled['UserEnc']),
    'ProductID': item_encoder.inverse_transform(X_resampled['ProductEnc']),
    'Rating': y_resampled,
    'UserEnc': X_resampled['UserEnc'],
    'ProductEnc': X_resampled['ProductEnc']
})

# بررسی توزیع جدید
print("\nتوزیع پس از نمونه‌افزایی:")
print(train_data_balanced['Rating'].value_counts())

# نمایش توزیع جدید با نمودار
plt.figure(figsize=(8, 5))
sns.countplot(x='Rating', data=train_data_balanced)
plt.title('توزیع امتیازات پس از نمونه‌افزایی')
plt.xlabel('امتیاز')
plt.ylabel('تعداد')
plt.show()

# نرمال‌سازی ستون Rating با MinMaxScaler
scaler = MinMaxScaler()
train_data_balanced['Rating_normalized'] = scaler.fit_transform(train_data_balanced[['Rating']])

# نمایش چند خط اول داده‌ها برای بررسی
print("\nداده‌ها پس از نرمال‌سازی:")
print(train_data_balanced.head())

In [4]:
import keras
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dot, Add, Dense

# تعداد کاربران و محصولات منحصربه‌فرد
n_users = train_data_balanced['UserEnc'].nunique()
n_items = train_data_balanced['ProductEnc'].nunique()

# اندازه‌ی بردار تعبیه
embedding_size = 50

# تعریف ورودی‌ها
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')

# لایه‌های تعبیه برای کاربر و کالا
user_embedding = Embedding(input_dim=n_users, output_dim=embedding_size,
                          name='user_embedding',
                          embeddings_initializer='glorot_normal',
                          embeddings_regularizer=keras.regularizers.l2(1e-6))(user_input)
item_embedding = Embedding(input_dim=n_items, output_dim=embedding_size,
                          name='item_embedding',
                          embeddings_initializer='glorot_normal',
                          embeddings_regularizer=keras.regularizers.l2(1e-6))(item_input)

# لایه‌های تعبیه برای بایاس کاربر و کالا
user_bias = Embedding(input_dim=n_users, output_dim=1,
                     name='user_bias',
                     embeddings_initializer='zeros')(user_input)
item_bias = Embedding(input_dim=n_items, output_dim=1,
                     name='item_bias',
                     embeddings_initializer='zeros')(item_input)

# مسطح کردن خروجی‌های تعبیه
user_vec = Flatten()(user_embedding)
item_vec = Flatten()(item_embedding)
user_bias_vec = Flatten()(user_bias)
item_bias_vec = Flatten()(item_bias)

# ضرب نقطه‌ای بین بردارهای تعبیه کاربر و کالا
dot_product = Dot(axes=1)([user_vec, item_vec])

# جمع بایاس‌ها و ضرب نقطه‌ای
output = Add()([dot_product, user_bias_vec, item_bias_vec])

# اعمال تابع فعال‌ساز sigmoid
output = Dense(1, activation='sigmoid', name='output')(output)

# تعریف مدل
model = Model(inputs=[user_input, item_input], outputs=output)

# تنظیم بهینه‌ساز با نرخ یادگیری اولیه
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# کامپایل مدل با تابع زیان binary_crossentropy
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['mae', 'mse'])

# نمایش خلاصه مدل
model.summary()

In [5]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
# آماده‌سازی داده‌ها برای آموزش
X_train_user = train_data_balanced['UserEnc'].values
X_train_item = train_data_balanced['ProductEnc'].values
y_train = train_data_balanced['Rating_normalized'].values

# تعریف callback‌ها برای بهبود آموزش
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001, verbose=1)

# آموزش مدل
history = model.fit(
    [X_train_user, X_train_item], y_train,
    batch_size=64,
    epochs=20,
    validation_split=0.2,
    callbacks=[checkpoint, reduce_lr],
    verbose=1
)

# نمایش نمودار خطا و معیارها
plt.figure(figsize=(12, 4))

# نمودار Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Binary Crossentropy)')
plt.legend()

# نمودار MAE
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend()

plt.tight_layout()
plt.show()

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
from keras.models import load_model

# فرض می‌کنیم مدل قبلاً آموزش دیده و در 'best_model.keras' ذخیره شده است
model = load_model('best_model.keras')

# فرض می‌کنیم user_encoder و item_encoder از مرحله‌ی کدگذاری در دسترس هستند
# همچنین scaler از مرحله‌ی نرمال‌سازی در دسترس است

# خواندن داده‌های اعتبارسنجی
valid_data = pd.read_csv('amazon_valid.csv')

# فیلتر کردن شناسه‌های نادیده در داده‌های اعتبارسنجی
valid_users = set(user_encoder.classes_)
valid_items = set(item_encoder.classes_)
valid_data = valid_data[valid_data['UserID'].isin(valid_users) & valid_data['ProductID'].isin(valid_items)]

# بررسی تعداد ردیف‌های باقی‌مانده
print(f"تعداد ردیف‌های اعتبارسنجی پس از فیلتر کردن: {len(valid_data)}")

# کدگذاری شناسه‌های کاربر و کالا
valid_data['UserEnc'] = user_encoder.transform(valid_data['UserID'])
valid_data['ProductEnc'] = item_encoder.transform(valid_data['ProductID'])

# نرمال‌سازی ستون rating
valid_data['Rating_normalized'] = scaler.transform(valid_data[['Rating']])

# آماده‌سازی داده‌های ورودی و هدف برای اعتبارسنجی
X_valid_user = valid_data['UserEnc'].values
X_valid_item = valid_data['ProductEnc'].values
y_valid = valid_data['Rating_normalized'].values
y_valid_true = valid_data['Rating'].values  # برچسب‌های اصلی (1 تا 5)

# پیش‌بینی مدل روی داده‌های اعتبارسنجی
y_valid_pred = model.predict([X_valid_user, X_valid_item], batch_size=64, verbose=1)

# تبدیل خروجی‌های نرمال‌شده به برچسب‌های باینری (4 و 5 مثبت، بقیه منفی)
# معادل‌سازی امتیازات نرمال‌شده به مقادیر اصلی: 4 و 5 در مقیاس اصلی به [0.75, 1] در مقیاس نرمال‌شده نگاشت می‌شوند
threshold = 0.75  # آستانه برای امتیازات 4 و 5 (قابل تنظیم برای بهینه‌سازی Precision)
y_valid_pred_binary = (y_valid_pred >= threshold).astype(int)  # پیش‌بینی‌های باینری
y_valid_true_binary = (y_valid_true >= 4).astype(int)  # برچسب‌های واقعی باینری

# محاسبه Precision
precision = precision_score(y_valid_true_binary, y_valid_pred_binary)
print(f"Precision روی مجموعه‌ی اعتبارسنجی: {precision:.4f}")

# بررسی توزیع پیش‌بینی‌ها
plt.figure(figsize=(8, 5))
plt.hist(y_valid_pred, bins=50, label='پیش‌بینی‌های نرمال‌شده')
plt.axvline(x=threshold, color='r', linestyle='--', label=f'آستانه = {threshold}')
plt.title('توزیع پیش‌بینی‌های مدل روی مجموعه‌ی اعتبارسنجی')
plt.xlabel('امتیاز نرمال‌شده پیش‌بینی‌شده')
plt.ylabel('تعداد')
plt.legend()
plt.show()

# بهینه‌سازی آستانه برای بهبود Precision
thresholds = np.arange(0.5, 0.9, 0.05)
precisions = []
for thresh in thresholds:
    y_pred_binary = (y_valid_pred >= thresh).astype(int)
    prec = precision_score(y_valid_true_binary, y_pred_binary)
    precisions.append(prec)
    print(f"Threshold: {thresh:.2f}, Precision: {prec:.4f}")

# رسم نمودار Precision نسبت به آستانه
plt.figure(figsize=(8, 5))
plt.plot(thresholds, precisions, marker='o')
plt.title('Precision در برابر آستانه‌های مختلف')
plt.xlabel('آستانه')
plt.ylabel('Precision')
plt.grid(True)
plt.show()

# انتخاب بهترین آستانه
best_threshold = thresholds[np.argmax(precisions)]
print(f"بهترین آستانه: {best_threshold:.2f}, Precision: {max(precisions):.4f}")

# آماده‌سازی پیش‌بینی‌ها برای مجموعه‌ی آزمون
test_data = pd.read_csv('amazon_test.csv')

test_data = test_data[test_data['UserID'].isin(valid_users) & test_data['ProductID'].isin(valid_items)]
print(f"تعداد ردیف‌های آزمون پس از فیلتر کردن: {len(test_data)}")
# کدگذاری شناسه‌های کاربر و کالا
test_data['UserEnc'] = user_encoder.transform(test_data['UserID'])
test_data['ProductEnc'] = item_encoder.transform(test_data['ProductID'])

# آماده‌سازی داده‌های ورودی برای آزمون
X_test_user = test_data['UserEnc'].values
X_test_item = test_data['ProductEnc'].values

# پیش‌بینی مدل روی داده‌های آزمون
y_test_pred = model.predict([X_test_user, X_test_item], batch_size=64, verbose=1)

# تبدیل پیش‌بینی‌ها به برچسب‌های باینری با بهترین آستانه
y_test_pred_binary = (y_test_pred >= best_threshold).astype(int)

In [7]:
# تبدیل امتیازات نرمال‌شده (بازه‌ی [0, 1]) به مقیاس اصلی (1 تا 5)
# فرض می‌کنیم MinMaxScaler روی مقادیر اصلی 1 تا 5 اعمال شده است
y_test_pred = scaler.inverse_transform(y_test_pred)

# گرد کردن امتیازات به نزدیک‌ترین عدد صحیح بین 1 تا 5
y_test_pred_rounded = np.clip(np.round(y_test_pred), 1, 5).astype(int)

# ایجاد DataFrame برای submission
submission = pd.DataFrame({
    'UserID': test_data['UserID'],
    'ProductID': test_data['ProductID'],
    'Rating': y_test_pred_rounded.flatten()
})
submission